# Notebook de la dimensión Fecha - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [1]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2. Extracción de datos desde la base de datos - Extract

In [3]:
from sqlalchemy import inspect

# Saber las columnas que hay en la base de datos OLTP
inspector = inspect(engine_fuente)
tablas = inspector.get_table_names()
print("Tablas de la base de datos:")
print(tablas)

Tablas de la base de datos:
['admin_interface_theme', 'areas_cliente', 'auth_user_user_permissions', 'clientes_coordinador', 'clientes_datosmensajero', 'clientes_mensajeroaquitoy_clientes', 'auth_permission', 'auth_group', 'auth_user_groups', 'auth_user', 'authtoken_token', 'ciudad', 'cliente', 'clientes_mensajeroaquitoy', 'area', 'django_migrations', 'departamento', 'clientes_usuarioaquitoy', 'django_admin_log', 'django_session', 'mensajeria_novedadesservicio', 'mensajeria_tiponovedad', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estado', 'mensajeria_tiposervicio', 'mensajeria_servicio', 'mensajeria_estadosservicio', 'mensajeria_tipovehiculo', 'sede', 'auth_group_permissions', 'django_content_type', 'tipo_cliente', 'mensajeria_destinoservicio', 'mensajeria_documentoasociado']


In [5]:
# Consulta de fechas mínimas y máximas de la base de datos OLTP
query_fechas_auth_user = """
SELECT 
    MIN(date_joined) AS min_date_joined,
    MAX(date_joined) AS max_date_joined,
    MIN(last_login) AS min_last_login,
    MAX(last_login) AS max_last_login
FROM auth_user;
"""

# Ejecutar la consulta
df_fechas_user = pd.read_sql(query_fechas_auth_user, engine_fuente)
print(df_fechas_user)

            min_date_joined           max_date_joined  \
0 2023-06-15 16:06:05+00:00 2024-08-30 20:48:57+00:00   

             min_last_login                   max_last_login  
0 2023-08-05 00:26:29+00:00 2024-08-31 19:41:33.430951+00:00  


## 3. Transformación de los datos - Transform

In [6]:
# Generar rango de fechas
fecha = pd.DataFrame({"general_date": pd.date_range(start="2023-06-01",end="2024-12-31",freq="d")})
fecha

,general_date
0,2023-06-01
1,2023-06-02
2,2023-06-03
3,2023-06-04
4,2023-06-05
...,...
575,2024-12-27
576,2024-12-28
577,2024-12-29
578,2024-12-30


In [7]:
# Agregar columnas a la dimensión fecha
fecha["id_fecha"] = fecha.index + 1
fecha["anio"] = fecha["general_date"].dt.year
fecha["date"] = fecha["general_date"].dt.date
fecha["anio_mes"] = fecha["general_date"].dt.to_period("M").astype(str)
fecha["mes"] = fecha["general_date"].dt.month
fecha["mes_str"] = fecha["general_date"].dt.month_name()
fecha["dia"] = fecha["general_date"].dt.day
fecha["dia_str"] = fecha["general_date"].dt.day_name()
fecha["trimestre"] = fecha["general_date"].dt.quarter
fecha["semana_anio"] = fecha["general_date"].dt.isocalendar().week
fecha

,general_date,id_fecha,anio,date,anio_mes,mes,mes_str,dia,dia_str,trimestre,semana_anio
0,2023-06-01,1,2023,2023-06-01,2023-06,6,June,1,Thursday,2,22
1,2023-06-02,2,2023,2023-06-02,2023-06,6,June,2,Friday,2,22
2,2023-06-03,3,2023,2023-06-03,2023-06,6,June,3,Saturday,2,22
3,2023-06-04,4,2023,2023-06-04,2023-06,6,June,4,Sunday,2,22
4,2023-06-05,5,2023,2023-06-05,2023-06,6,June,5,Monday,2,23
...,...,...,...,...,...,...,...,...,...,...,...
575,2024-12-27,576,2024,2024-12-27,2024-12,12,December,27,Friday,4,52
576,2024-12-28,577,2024,2024-12-28,2024-12,12,December,28,Saturday,4,52
577,2024-12-29,578,2024,2024-12-29,2024-12,12,December,29,Sunday,4,52
578,2024-12-30,579,2024,2024-12-30,2024-12,12,December,30,Monday,4,1


In [8]:
# Reordenar columnas
fecha = fecha[["id_fecha", "anio", "date", "anio_mes", "mes", "mes_str", "dia", "dia_str", "trimestre", "semana_anio"]]
fecha

,id_fecha,anio,date,anio_mes,mes,mes_str,dia,dia_str,trimestre,semana_anio
0,1,2023,2023-06-01,2023-06,6,June,1,Thursday,2,22
1,2,2023,2023-06-02,2023-06,6,June,2,Friday,2,22
2,3,2023,2023-06-03,2023-06,6,June,3,Saturday,2,22
3,4,2023,2023-06-04,2023-06,6,June,4,Sunday,2,22
4,5,2023,2023-06-05,2023-06,6,June,5,Monday,2,23
...,...,...,...,...,...,...,...,...,...,...
575,576,2024,2024-12-27,2024-12,12,December,27,Friday,4,52
576,577,2024,2024-12-28,2024-12,12,December,28,Saturday,4,52
577,578,2024,2024-12-29,2024-12,12,December,29,Sunday,4,52
578,579,2024,2024-12-30,2024-12,12,December,30,Monday,4,1


In [9]:
from datetime import datetime

# Agregar columna de fecha de carga
fecha["fecha_carga"] = datetime.now().date()

## 4. Guardado de la dimensión - Load

In [10]:
# Guardar como archivo CSV
fecha.to_csv("dim_fecha.csv", index=False)

In [11]:
# Cargar en la bodega de datos OLAP
fecha.to_sql("dim_fecha", engine_bodega, if_exists="replace", index=False)

580

In [12]:
# Verificación del archivo
df_verificacion = pd.read_csv("dim_fecha.csv")
print(df_verificacion.head())

   id_fecha  anio        date anio_mes  mes mes_str  dia   dia_str  trimestre  \
0         1  2023  2023-06-01  2023-06    6    June    1  Thursday          2   
1         2  2023  2023-06-02  2023-06    6    June    2    Friday          2   
2         3  2023  2023-06-03  2023-06    6    June    3  Saturday          2   
3         4  2023  2023-06-04  2023-06    6    June    4    Sunday          2   
4         5  2023  2023-06-05  2023-06    6    June    5    Monday          2   

   semana_anio fecha_carga  
0           22  2025-06-05  
1           22  2025-06-05  
2           22  2025-06-05  
3           22  2025-06-05  
4           23  2025-06-05  
